In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# read text file into pandas DataFrame
Leaderboard = pd.read_csv('data/PAKDD2010_Leaderboard_Submission_Example.txt', header=None, delim_whitespace=True)
#df = pd.read_table('..\data\PAKDD2010_Leaderboard_Data.txt',header = None) # delimiter=" ")

#df = pd.read_fwf('..\data\PAKDD2010_Leaderboard_Data.txt')

In [ ]:
Modeling_data = pd.read_csv('data/PAKDD2010_Modeling_Data.txt', encoding= "ISO-8859-1", header= None, delimiter='\t')
Modeling_data.shape

In [ ]:
prediction_data = pd.read_csv('data/PAKDD2010_Prediction_Data.txt', encoding= "ISO-8859-1", header= None, delimiter='\t')
prediction_data.shape

In [ ]:
pd.read_excel('data/PAKDD2010_VariablesList.XLS', index_col=0)

In [5]:
import xlrd
variable_list = pd.read_excel('data/PAKDD2010_VariablesList.XLS', index_col=0)
variable_list

ModuleNotFoundError: No module named 'xlrd'